# Exercise 1: Agent Hello World

**Goal:** Create a simple AI agent with one tool function using the OpenAI Agents SDK.

**Timebox:** 15 minutes

## What You'll Learn

- How to create an agent with system instructions
- How to define and register tool functions
- How to run the agent and see tool calls in action

## Setup

Import required libraries and load environment variables.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from rich.console import Console

# Load environment variables
load_dotenv()

console = Console()
console.print("[green]✓[/green] Environment loaded")

## Step 1: Define a Tool Function

Create a simple `get_weather()` function that returns weather data.

**TODO:** Implement the function to return mock weather data.

In [ ]:
def get_weather(location: str) -> str:
    """
    Tool function: Get current weather for a location.
    
    In a real application, this would call a weather API.
    For this exercise, we return mock data.
    """
    # TODO: Return mock weather data
    # Example: return f"The weather in {location} is sunny, 72°F"
    pass

## Step 2: Define the Tool Schema

Create the tool schema that describes the function to the LLM.

**TODO:** Define the tools array with the get_weather function schema.

In [ ]:
# TODO: Define tools array
tools = [
    # Add your tool schema here
]

## Step 3: Initialize OpenAI Client

Create the OpenAI client instance.

In [ ]:
# TODO: Initialize OpenAI client
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Step 4: Run the Agent

Execute the agent with a query that will trigger the tool.

**TODO:** Implement the agent execution logic.

In [ ]:
# TODO: Define the query
query = "What's the weather like in San Francisco?"

console.print(f"[yellow]Query:[/yellow] {query}\n")

# TODO: Make API call with tools
# TODO: Handle tool calls if present
# TODO: Get final response

console.print("⚠️  Implement the TODOs above!")
console.print("   Check the solution notebook for reference")

## Stretch Goal

- Add a second tool function (e.g., `get_time()`)
- Modify the query to use both tools
- Observe how the agent orchestrates multiple tool calls